In [ ]:
from icecube import dataio, icetray , dataclasses 


In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)   # tüm kolonlar


In [ ]:
from collections.abc import Iterable

In [ ]:
GCD_PATH = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
data_file = dataio.I3File(GCD_PATH)
data_file.pop_frame()
data_file.pop_frame()
detector_status = data_file.pop_frame()

In [ ]:
# dir(detector_status)

In [ ]:
print(type(detector_status))
print("Stop:", detector_status.Stop)   
print("Key's:", list(detector_status.keys()))


In [ ]:
# Small helper: print the type and a short summary for each key in the frame
for key in detector_status.keys():
    obj = detector_status[key]
    print("="*60)
    print(f"Key: {key}")
    print("Type:", type(obj))
    try:
        print("dir(obj) first 10:", dir(obj)[:10])
    except Exception as e:
        print("dir(obj) could not be obtained:", e)

# - DetectorStatus.I3Calibration
###### This may be included in G or C frame. Check it

# - DetectorStatus.StartTime and DetectorStatus.EndTime
###### This may be included in G or C frame. Check it

# - DetectorStatus.I3DetectorStatus

In [ ]:
det_obj = detector_status["I3DetectorStatus"]

print("Type:", type(det_obj))




In [ ]:
# --- public attribute's ---
attrs = [a for a in dir(det_obj) if not a.startswith("_")]
print("\nPublic attrs:")
for a in attrs:
    print("  -", a)

In [ ]:

rows = []
for a in attrs:
    val = getattr(det_obj, a)
    try:
        n = len(val)        
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(val).__name__,
        "len": n,
        "repr_sample": repr(val)[:80],   
    })

det_status_i3_df = pd.DataFrame(rows)
det_status_i3_df


## -- dom_status

In [ ]:
dom_status_map = det_obj.dom_status

print("Type:", type(dom_status_map))
print("Number of entries:", len(dom_status_map))

# peek at one exam ple
first_key, first_val = next(iter(dom_status_map.items()))
print("\nExample key:", first_key)
print("Example value type:", type(first_val))
print("Example value repr:\n", first_val)


In [ ]:

dom_attrs = [a for a in dir(first_val) if not a.startswith("_")]

rows = []
for a in dom_attrs:
    v = getattr(first_val, a)
    try:
        n = len(v)
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(v).__name__,
        "len": n,
        "repr": repr(v)[:80],  
    })

dom_status_one_df = pd.DataFrame(rows).sort_values("attr")
dom_status_one_df


In [ ]:
import inspect

example_status = first_val   # this is the I3DOMStatus for OMKey(1,1,1)

per_dom_attrs = []
for a in dir(example_status):
    if a.startswith("_"):
        continue
    # only attributes starting with a lowercase letter (skip enum constants like CPU, SPE, etc.)
    if not a[0].islower():
        continue

    v = getattr(example_status, a)
    # skip methods / functions
    if inspect.ismethod(v) or inspect.isfunction(v):
        continue

    per_dom_attrs.append(a)

print("Per-DOM attributes:")
for a in per_dom_attrs:
    print("  -", a)


In [ ]:
import pandas as pd

rows = []

for omkey, status in dom_status_map.items():
    row = {
        "string": omkey.string,
        "om": omkey.om,
        "pmt": getattr(omkey, "pmt", None),
    }

    for a in per_dom_attrs:
        v = getattr(status, a)

        # enum'ları okunur stringe çevir
        if type(v).__name__ in ["DOMGain", "LCMode", "TrigMode", "OnOff", "CableType"]:
            row[a] = v.name
        else:
            row[a] = v

    rows.append(row)

dom_status_df = pd.DataFrame(rows)
dom_status_df.head()


In [ ]:
n_unique = dom_status_df.nunique()

const_cols = n_unique[n_unique == 1].index.tolist()
vary_cols  = n_unique[n_unique > 1].index.tolist()

print("Columns that are constant across all DOMs:")
for c in const_cols:
    print("  -", c)

print("\nColumns that vary between DOMs:")
for c in vary_cols:
    print("  -", c)


### 3.3.5 trigger_status

In [ ]:

trig_status_map = det_obj.trigger_status

print("Type:", type(trig_status_map))
print("Number of entries:", len(trig_status_map))

# peek at one example
first_tkey, first_tval = next(iter(trig_status_map.items()))
print("\nExample TriggerKey:", first_tkey)
print("Example value type:", type(first_tval))
print("Example value repr:\n", first_tval)


In [ ]:


first_tkey, first_tval = next(iter(trig_status_map.items()))

trig_attrs = [a for a in dir(first_tval) if not a.startswith("_")]

rows = []
for a in trig_attrs:
    v = getattr(first_tval, a)
    try:
        n = len(v)
    except TypeError:
        n = None
    rows.append({
        "attr": a,
        "type": type(v).__name__,
        "len": n,
        "repr": repr(v)[:80],
    })

trig_status_one_df = pd.DataFrame(rows).sort_values("attr")
trig_status_one_df


In [ ]:

first_tkey, first_tval = next(iter(trig_status_map.items()))

per_trig_attrs = []
for a in dir(first_tval):
    if a.startswith("_"):
        continue
    if not a[0].islower():
        continue

    v = getattr(first_tval, a)
    if inspect.ismethod(v) or inspect.isfunction(v):
        continue

    per_trig_attrs.append(a)

print("Per-trigger attrs:")
for a in per_trig_attrs:
    print("  -", a)


In [ ]:
rows = []

for tkey, status in trig_status_map.items():
    row = {
        # Info from the TriggerKey
        "source":    tkey.source,      # e.g. IN_ICE, GLOBAL, ICETOP
        "type":      tkey.type,        # e.g. SIMPLE_MULTIPLICITY
        "config_id": tkey.config_id,   # e.g. 1006
    }

    # Make enums human-readable strings (source/type are usually enums)
    for k in ["source", "type"]:
        v = row[k]
        if hasattr(v, "name"):
            row[k] = v.name

    # Per-trigger attributes
    for a in per_trig_attrs:
        v = getattr(status, a)

        # For complex maps like readout_settings / trigger_settings,
        # convert to something printable; try dict, fall back to repr
        if hasattr(v, "items"):
            try:
                row[a] = dict(v.items())
            except Exception:
                row[a] = repr(v)
        else:
            row[a] = v

    rows.append(row)

trig_status_df = pd.DataFrame(rows)
trig_status_df


In [ ]:
# pick one trigger, e.g. the first row
example_key, example_status = next(iter(trig_status_map.items()))

print("TriggerKey:", example_key)
print("\ntrigger_name:", example_status.trigger_name)

# --- trigger_settings: use string repr instead of .items() ---
print("\ntrigger_settings (as text):")
print(str(example_status.trigger_settings))

# If you want it line-by-line:
print("\ntrigger_settings (line by line):")
for line in str(example_status.trigger_settings).splitlines():
    print(" ", line)

# --- readout_settings: this one *does* support .items() ---
print("\nreadout_settings (per subdetector):")
for subd, cfg in example_status.readout_settings.items():
    print(" ", subd, "->", cfg)


# - DetectorStatus.I3Geometry
###### This may be included in G or C frame. Check it

# - DetectorStatus.SPEAbove
###### This may be included in G or C frame. Check it

# - DetectorStatus.SPEScalingFactors
###### This may be included in G or C frame. Check it

# - DetectorStatus.I3ModuleGeoMap
###### This may be included in G or C frame. Check it

# - DetectorStatus.I3OMGeoMap
###### This may be included in G or C frame. Check it

# - DetectorStatus.Subdetectors
###### This may be included in G or C frame. Check it

# - DetectorStatus.BadDomsList

In [ ]:
bad_doms = detector_status["BadDomsList"]

print("Type:", type(bad_doms))



In [ ]:

try:
    print("Number of entries:", len(bad_doms))
except TypeError:
    print("len() not defined")

print("\nFirst few entries:")
for i, omkey in enumerate(bad_doms):
    print("  ", omkey)
    if i >= 9:   
        break

# - DetectorStatus.BadDomsListSLC

In [ ]:
bad_doms_slc = detector_status["BadDomsListSLC"]

print("Type:", type(bad_doms_slc))



In [ ]:
try:
    print("Number of entries:", len(bad_doms_slc))
except TypeError:
    print("len() not defined")

print("\nFirst few entries:")
for i, omkey in enumerate(bad_doms_slc):
    print("  ", omkey)
    if i >= 9:
        break